# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np 
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []


        ############ New definitions ##############
        self.max_gram_num = 5
        self.all_letters_length = 28
        self.guessed_letters_mask = np.zeros(self.all_letters_length ,dtype=bool)
        self.weights = np.array([1, 2, 5, 10, 100])

        ############ New definitions ##############
        self.gram1 = np.zeros(self.all_letters_length)
        self.gram2 = np.zeros(tuple([self.all_letters_length]*2))
        self.gram3 = np.zeros(tuple([self.all_letters_length]*3))
        self.gram4 = np.zeros(tuple([self.all_letters_length]*4))
        self.gram5 = np.zeros(tuple([self.all_letters_length]*5))

        for word in self.full_dictionary:
            self.get_grams(word)

    ############ Define new function ##############
    def get_grams(self, word):
        word = '{' + word + '|'
        self.get_gram_1(word)
        self.get_gram_n(self.gram2, word, 2)
        self.get_gram_n(self.gram3, word, 3)
        self.get_gram_n(self.gram4, word, 4)
        self.get_gram_n(self.gram5, word, 5)

    def get_gram_1(self, word):
        for char in set(word):
            a = ord(char) - ord('a')
            if 0 <= a < self.all_letters_length:
                self.gram1[a] += 1

    def get_gram_n(self, ngram, word, n):
        index = 0
        while index + n - 1 < len(word):
            ngram_indices = [ord(word[index + i]) - ord("a") for i in range(n)]
            if all(0 <= idx < self.all_letters_length for idx in ngram_indices):
                ngram[tuple(ngram_indices)] += 1
            index += 1


    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
    

    
    ############# Define new function ##############
    def cal_letter_prob(self, clean_word, gram_length):
        '''
        - Calculate the score for each letter (26) based on the occurrence of grams (1 to 5) in self.current_dictionary;
        - The function will be called whenever 'guess' function is called
        '''
        prob = np.zeros(self.all_letters_length)

        for idx in range(len(clean_word)-gram_length+1):
            gram = clean_word[idx:idx+gram_length]
            if gram.count('_') == 1:
                prob += self.cal_letter_gram_prob(gram)

        return prob
    
    ############# Define new function ##############
    def cal_letter_gram_prob(self, gram):
        '''
        - Calculate the probability of each letter (26) based on the occurrence of grams (1 to 5) in self.current_dictionary;
        - The function will be called whenever 'cal_letter_prob' function is called
        '''
        occurrence = np.zeros(self.all_letters_length)

        if len(gram) > 1:
            for i in range(self.all_letters_length):
                if self.guessed_letters_mask[i]:
                    continue

                temp = gram.replace('_', chr(ord('a') + i))
                # Can be improved
                ngram = getattr(self, f'gram{len(gram)}', None)
                if ngram is not None:
                    idx = [ord(char)-ord('a') for char in temp]
                    occurrence[i] = ngram[tuple(idx)]
        
        all_no_guessed = sum(occurrence[~self.guessed_letters_mask])

        if all_no_guessed < 1:
            occurrence[~self.guessed_letters_mask] = self.gram1[~self.guessed_letters_mask]
            all_no_guessed = sum(self.gram1[~self.guessed_letters_mask])

        return occurrence / all_no_guessed
    

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        word = '{' + word + '|'
        clean_word = word.replace(' ', '') 

        # Initialize probability array
        probability = np.zeros(self.all_letters_length)

        for i in range(self.max_gram_num):
            temp_prob = self.cal_letter_prob(clean_word=clean_word, gram_length=i+1) * self.weights[i]
            probability += temp_prob
    
        guess_letter = chr(probability.argmax() + ord('a'))
        self.guessed_letters_mask[probability.argmax()] = True


        return guess_letter




    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary


        ############ Reset numbers ##############
        self.guessed_letters_mask = np.zeros(28, dtype=bool)
        self.guessed_letters_mask[26] = True 
        self.guessed_letters_mask[27] = True  

        
                 
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
api = HangmanAPI(access_token="94ffa4500e5260ab2a054c56aca4e5", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [9]:
success=0 
for i in range(50):
    
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 1192 practice games out of an allotted 100,000. practice success rate so far = 0.402
run 1193 practice games out of an allotted 100,000. practice success rate so far = 0.402
run 1194 practice games out of an allotted 100,000. practice success rate so far = 0.402
run 1195 practice games out of an allotted 100,000. practice success rate so far = 0.403
run 1196 practice games out of an allotted 100,000. practice success rate so far = 0.403
run 1197 practice games out of an allotted 100,000. practice success rate so far = 0.403
run 1198 practice games out of an allotted 100,000. practice success rate so far = 0.402
run 1199 practice games out of an allotted 100,000. practice success rate so far = 0.403
run 1200 practice games out of an allotted 100,000. practice success rate so far = 0.403
run 1201 practice games out of an allotted 100,000. practice success rate so far = 0.404
run 1202 practice games out of an allotted 100,000. practice success rate so far = 0.404
run 1203 practice gam

In [8]:
api.my_status()

[1191, 0, 0, 478]